## INST326 Project 01: 21 Card Game

#### David Lin (replace my name with yours on this line and in the notebook name)

Refer to the game description included with the assignment for a description of the card game 21 and rules of play we will use for this project.

Place any opening comments here

### Imports

In [1]:
import random
import tkinter as tk

from tkinter import ttk

Note on the imports above
from tkinter import * imports everything from tkinter directly into the main program. This is generally bad practice for several reasons including the potential for variable naming conflicts. It is provided here for compatibility with code examples from the BroCode videos. Your code would be better if you preceeded all calls to tkinter methods with tk. For example, window = tk.Tk() instead of window = Tk().

import tkinter as tk also imports everything, but keeps all of the functionality separate from the main program space. This avoids potential naming conflicts, among other things. This is the preferred way to do it and it is used in the Cardbutton class below.

from tkinter import ttk is technically redundant, but for a specific purpose. The ttk class in tkinter represents a significant upgrade in many of the methods originally included in tkinter. However, many applications were already running with regular tk functionality when ttk was released. While ttk is substantively better, the original tk was retained for backward compatibility. If you do more programming in tkinter beyond this course you should learn and program with ttk from the start. It is imported here specifically to distinguish between old tk and new ttk widgets in the code.


### Cardbutton Class

This class combines a built in tkinter class - Button() - and modifies it to add the attributes we want for our card buttons. This is called inheritance. Since we have not really covered this yet, this class is being provided for you to use in your program. Please take a few moments to review it so you understand what is going on in the class.

In [2]:
# creates a custom button that inherits all the features of a tk.Button() and adds card attributes
class Cardbutton(tk.Button):
    def __init__(self, master=None, suit=None, rank=None, value=None): # the arguments on this line are inbound, meaning we pass them when we instantiate the object
        super().__init__(master) # on this line we call the __init__ method of tk.Button and pass the master attribute to it. This gives us all the button attributes and functionality
        self.config(bg = 'green')

        # the rest of the attributes in the main __init__ method are card attributes from our former card class
        self.suit = suit
        self.rank = rank # face rank of the card
        self.value = value # integer value of the card
        self.name = f"{rank}_of_{suit}"
        self.face = f"images\\{rank} of {suit}.png" # relative address
        self.back = "images\\BackOfCard.png"  # relative address
        self.suit_val = suits.index(suit)
        self.facetk = tk.PhotoImage(file = self.face) # calls the tk.PhotoImage method which creates an image object
        self.backtk = tk.PhotoImage(file = self.back)
        self.display = self.facetk # sets the default display
        self.hand = None # an attribute that allows cards to be assigned to specific hands


        # Bind mouse events
        self.bind("<Enter>", self.on_hover)
        self.bind("<Leave>", self.on_leave)

    def on_hover(self, event): # change the background and card image when the cursor hovers over it
        self.config(bg="lightblue")  

    def on_leave(self, event): # change back when not hovering
        self.config(bg="green", image=self.display)  # Restore original color
 
    def ace_toggle(self): # toggle the value of an ace
        if self.rank == 'ace':
            if self.value == 11:
                self.value = 1
            else:
                self.value = 11


#### Initialize Game Variables

In [3]:
# initialize your game variables here by adding your suits, ranks and values created in lab 01.
suits = ['clubs', 'diamonds', 'hearts', 'spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 11]
nplayers = 1 # for this project we are limiting the code to one player.

Remember, Python is a dynamically-typed language. We are not *declaring* the variables above. Rather, we are initializing or instantiating them because they are referenced in the code that follows.

### Game Class

In the cell below create your game class. You can refer to labs 1 and 2, and reuse code created there, or in the code examples used in class. Your class shoud start with an __init__ method. It should also include definitions of the following methods: make_deck, deal, hit, stay, and call. (more on these in a bit.) You should also include a method to calculate the score for a given hand. You may include the code to shuffle the deck in your make_deck method so it returns a shuffled deck, or you may handle it separately as we have in the class examples. Be sure to document your code with comments.

In [4]:
# Create your game class here
class Game():
    """So I included a deck as every game needs a deck, and we only have 1 player so an empty hand for player and the dealer. Then I had columns as I am incrementing them with the grid function.
    Then I had the scores as all the scores start at 0"""
    def __init__(self):
        self.deck = self.make_deck()
        self.dealer_hand = []
        self.player_hand = []
        self.player_card_column = 0
        self.dealer_card_column = 0
        self.player_score = 0
        self.dealer_score = 0
        
    #Method to make the deck
    def make_deck(self):
        deck = []
        for suit in suits:
            for rank, value in zip(ranks, values):
                card_button = Cardbutton(suit = suit, rank = rank, value = value)
                deck.append(card_button)
        random.shuffle(deck)
        self.deck = deck
                
#Method to deal the deck, it deals dealer then player then dealer then player.
    def deal(self):
        for i in range(2):
            self.dealer_hand.append(self.deck.pop())
            self.player_hand.append(self.deck.pop())
    
  

    #How I calculate the scores with the number of aces and score set to 0 then using the rank and values to calculate the score for each hand
    def scores(self, hand):
        score = 0
        num_aces = 0

        for card in hand:
            if card.rank in ['jack', 'queen', 'king']:
                score += 10
            elif card.rank == 'ace':
                num_aces += 1
            else:
                score += card.value

    #How ace calculations work as ace can be 1 or 11
        for _ in range(num_aces):
            if score + 11 <= 21:
                score += 11
            else:
                score += 1

        return score
    """Basically I have two ways of hitting, the way the player hits which it will always hit if you click hit and then there is the way that dealer hits which only be appliable if the dealer is below 17. """
    def player_hit(self):
        hit_card = self.deck.pop()
        self.player_hand.append(hit_card)
        self.player1_score = self.scores(self.player_hand)
        for card in self.player_hand:
            if card.rank == "ace" and self.player1_score > 21:
                card.ace_toggle()
                self.player1_score = self.scores(self.player_hand)
                break
        player1_label.config(text = f"test\nScore: {self.scores(self.player_hand)}")
        self.player_card_column += 1
        imageFront = tk.PhotoImage(file = card.face)
        player_card = tk.Button(root, image = imageFront)
        player_card.photo = imageFront
        player_card.grid(row = 1, column = self.player_card_column, padx = "20px", pady = "20px")
    
    #How the dealer deals
    def dealer_hit(self):
        if self.scores(self.dealer_hand) < 17:
            hit_card = self.deck.pop()
            self.dealer_hand.append(hit_card)
            self.dealer1_score = self.scores(self.dealer_hand)

            for card in self.dealer_hand:
                if card.rank == "ace" and self.dealer1_score > 21:
                    card.ace_toggle()

            self.dealer_card_column += 1
            card = self.dealer_hand[-1]
            imageFront = tk.PhotoImage(file=card.face)
            dealer_card = tk.Button(root, image=imageFront)
            dealer_card.photo = imageFront
            dealer_card.grid(row=0, column=self.dealer_card_column, padx="20px", pady="20px")
        
        self.dealer1_score = self.scores(self.dealer_hand)

        dealer_label.config(text=f"test\nScore: {self.dealer1_score}")
    
    #I thought that these functions would be easier to do outside the class so I just passed them
    def stay(self):
        pass
    def call(self):
        pass


Class notes
(add any additional notes you want relative to your game class in this cell. You do not need to repeat information included in comments within the code.)

### Main Execution

Write your main execution code in the code cell below. Be sure to document your code with comments. You may reuse and/or modify code from Lab 1, Lab 2, and/or the class examples as needed. DO NOT copy code from online examples or use generative AI like ChatGPT to generate your code. You may follow the examples in the BroCode video.

#### Instructions
1. Create a window and name it 'Project 01: Blackjack Game'. Refer to lab 02.
2. Instantiate a game and name it mygame.
3. Create a shuffled deck named mydeck using the make_deck method from your new game instance, mygame. Remember that every card in your deck is actually a tkinter button. You have instantiated them but have not added them to the window yet.
4. Create two hands, one for player1 and one for the dealer, and deal the hands using the deal method you created in your mygame instance. You may create the hands in the main code cell, or you may create a method within your game class - it's your choice (method in the game class might be better). Set the 'hand' attribute of each card to reflect the hand it belongs to. (Hint: if you create a method, you might consider adding two attributes to your class: player1_hand and dealer_hand. Set them to empty lists [] by default.)
5. Either during the deal or after, set the 'display' attribute of the second card dealt to the dealer to 'backtk'. Take a look at the Cardbutton class and notice that the display attribute is set to 'facetk' by default.  <font color = red>**Change to the display requirements: All player cards to be displayed face-up. Second dealer card to be displayed face-down.**</font>  
6. Create a method in your game class to calculate the score (sum of card values) for each hand - player1 and dealer. Hint: you might create player1_score and dealer_score attributes in your game class. Set the to 0 by default.
7. Create two tkinter labels, one for player1 and 1 for the dealer. The text for each label should display as two lines (include '\n' where you want the line break.) The first line should display the player or players name (you could include an input statement to get the player's name). The second line should include the player's score. EXCEPT: do not show the dealers score until after the 'Call' button is pressed.
8. Create four regular buttons (not Cardbuttons) using the tk.Button() method: Hit, Stay, Call, and Quit. The command= attribute should reference the respective methods you created (or will create) in the game class. See Below.
8a. hit button / hit method. the hit method should draw a card from the deck and add it to the player's (or dealer's) hand, updating the card's 'hand' attribute. The player's (or dealer's) score should be updated to reflect the value of the additional card. The players (or dealer's) label should be updated with the new score using the label.config() method. The new card should be added to the window - face up.
8b. stay button / stay method. the stay button is clicked when player1 has taken all the hit cards they want. the stay method then initiates an automated 'hit' sequence for the dealer according to the following rule: if dealer_score < 17, hit; else stay. Dealer's score and cards in the window are updated with each hit.
8c. call button / call method. Onece the dealer's hand has been updated in the stay method, the player presses the call button to compute and display all scores, show all cards, determine a winner, and display the winner message in the window. If both the player and dealer scores are greater than 21, there is no winner. if the scores are the same it is a tie. Else, the hand with the closest score <= 21 is the winner. If it is the player, display the message "Winner winner, chicken dinner!" in the window. If the dealer wins, display "House wins." in the window.
8d. quit button: the command= attribute should be: command=window.destroy. That will close the window and end the program.
9. Add all your widgets to the window. We will be using tkinter grid layout (refer to lab 2). add 20px of padding (x and y) to each item.
9a. add player1 label to row 1, column 0. Add dealer label to row 2, column 0.
9b. Create a loop to add your cards. player cards in row 1, dealer cards in row 2.
9c. Add your four play buttons (hit, stay, call, and quit) to row 3 and columns 0, 1, 2, and 3, respectively.
10. ~Add a 'peek' method that allows the player to see the cards in their hand. You may do this in one of two ways - your choice.~  
~10a. Display the player's cards face up in a new window (refer to Lab 02) *OR:*  
10b. Modify the Cardbutton.on_hover() method as follows: add image=self.facetk to the self.config() statement (refer to the on_leave() method for format. Then place the self.config() statement inside an if statement that only executes self.config() if self.hand == 'player1'. else, it's just self.config(bg="lightblue"). This conditional loop prevents the player from peeking at the dealer's cards.~  <font color = red>*Since players hands will be displayed face-up, the functionality in step 10 is no longer needed.*</font>  
11. Make sure your last line is: window.mainloop()
12. Document your code with comments.



One final note: When developing your tkinter code in Jupyter notebook, crashes tend to wreak havok on the kernal and further functioning of your program. If you start getting error messages for code you think should work, try restarting the kernal and clearing all output under the Kernal menu item above. Sometimes it takes a few seconds, so wait for all your old output and error messages to disappear.

In [5]:
# Write your main execution here
#Setting up my background and window 

#Instruction 1
root = tk.Tk()
root.title("Project 01: Blackjack Game")
root.geometry("1000x800")
root.config(bg = 'green')

# instruction 2
mygame = Game()

# instruction 3

mygame.make_deck()
# instruction 4
mygame.player_hand
mygame.dealer_hand
mygame.deal()
for card in mygame.player_hand:
    print(card.value)

mygame.dealer_hand[1].display = mygame.dealer_hand[1].backtk
print(mygame.scores(mygame.player_hand))

#Instruction 7
player1_label = tk.Label(root, text = f"Player \n Score:  {mygame.scores(mygame.player_hand)}", font = ("Gill Sans", "25"), bg = "green")
player1_label.grid(row = 1, column = 0)
dealer_label = tk.Label(root, text = "Dealer \n Score:  ", font = ("Gill Sans", "25"), bg = "green")
dealer_label.grid(row = 0, column = 0)

#Instruction 8
hit_button = tk.Button(root, text= "Hit", command= mygame.player_hit)
hit_button.grid(column = 0, row = 2)



#Making my call button, basically calculates the score and then prints out if you won or not
def call():
    results = tk.Label(root, text = "", bg = "Green")
    #print(mygame.scores(mygame.player_hand))
    if mygame.scores(mygame.player_hand) > 21 and mygame.scores(mygame.dealer_hand) > 21:
        results.config(text = "No Winner")
    elif mygame.scores(mygame.player_hand) > 21:
        results.config(text = "House Wins")
    elif mygame.scores(mygame.dealer_hand) > 21:
        results.config(text = "Winner winner, chicken dinner!")
    elif mygame.scores(mygame.player_hand) > mygame.scores(mygame.dealer_hand) and mygame.scores(mygame.player_hand) < 22:
        results.config(text = "Winner winner, chicken dinner!")
    elif mygame.scores(mygame.dealer_hand) > mygame.scores(mygame.player_hand) and mygame.scores(mygame.dealer_hand) < 22:
        results.config(text = "House Wins")
    elif mygame.scores(mygame.dealer_hand) == mygame.scores(mygame.player_hand):
        results.config(text = "Tie")
    results.grid(row = 5, column = 2)
call_button = tk.Button(root, text = "Call", command = call)
call_button.grid(column = 2, row = 2)


#The stay button activates the dealer hit and then updates the cardbuttons
def stay():
    mygame.dealer1_score = mygame.scores(mygame.dealer_hand)

    dealer_label.config(text=f"test\nScore: {mygame.dealer1_score}")
    while mygame.scores(mygame.dealer_hand) < 17:
        mygame.dealer_hit()
    mygame.dealer_card_column = 0
    for card in mygame.dealer_hand:
        mygame.dealer_card_column += 1
        front = tk.PhotoImage(file = card.face)
        dealer_card = tk.Button(root, image=front)
        dealer_card.photo = front
        dealer_card.grid( column = mygame.dealer_card_column, row = 0, padx = "20px", pady = "20px")


    
    
    
    
#Actually making the button    
stay_button = tk.Button(root, text = "Stay", command = stay)
stay_button.grid(column = 1, row = 2)

#Closes the window
quit_button = tk.Button(root, text  = "Quit", command = root.destroy)
quit_button.grid(column = 3, row = 2)

#The code below just makes the initial card button and images on the screen
for card in mygame.player_hand:
    mygame.player_card_column += 1
    front = tk.PhotoImage(file = card.face)
    player_card = tk.Button(root, image=front)
    player_card.photo = front
    player_card.grid( column = mygame.player_card_column, row = 1, padx = "20px", pady = "20px")

first_card = 0    
for card in mygame.dealer_hand:
    mygame.dealer_card_column += 1
    image = card.display
    if first_card != 0:
        image = card.backtk
        
    dealer_card = tk.Button(root, image=image)
    dealer_card.photo = image
    dealer_card.grid( column = mygame.dealer_card_column, row = 0, padx = "20px", pady = "20px")
    first_card += 1
    

    



root.mainloop()

7
11
18


Main execution notes.
(add any additional notes here. Do not repeat what you have already documented in the code.)

### Extra Credit

10 points extra credit will be awarded for converting all code to work with the import statement import tkinter as tk instead of 'from tkinter import * '. To claim the extra credit:
1. Make the necessary code edits
2. Comment out the 'from tkinter import * ' statement in the imports cell at the top of this notebook.
3. Enter 'My code works. I claim 10 points of extra credit.' in the cell below.

My code works. I claim 10 points of extra credit.

#### A Helpful Check

The following code is borrowed from the in-class example and may be used to take a look at the attributes of your card objects after you create your deck. See the GUI example for additional notes.

In [6]:
mycard = deck.pop()
print(mycard)
print()
print(vars(mycard))
print()
print(mycard.name)

NameError: name 'deck' is not defined